In [13]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [14]:
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8,
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

### Experiment 1: Train Logistic Regression Classifier

In [16]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)

y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



### Experiment 2: Train Random Forest Classifier

In [17]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)

y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       270
           1       0.95      0.67      0.78        30

    accuracy                           0.96       300
   macro avg       0.96      0.83      0.88       300
weighted avg       0.96      0.96      0.96       300



### Experiment 3: Train XGBoost

In [18]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)

y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



### Experiment 4: Handle class imbalance using SMOTETomek and then Train XGBoost

In [19]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619], dtype=int64))

In [20]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)

y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.81      0.83      0.82        30

    accuracy                           0.96       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.96      0.96      0.96       300



### Track Experiment using MLFlow

In [21]:
models = [
    (
        "Logistic Regression",
        LogisticRegression(C=1, solver='liblinear'),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest",
        RandomForestClassifier(n_estimators=30, max_depth=3),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [22]:
reports = []

for model_name, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [23]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [24]:
mlflow.set_tracking_uri('http://localhost:8080')
mlflow.set_experiment("Anomaly Detection1")

for i, element in enumerate(models):
    model_name = element[0]
    model = element[1]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):
        mlflow.log_param("model", model_name)
        mlflow.log_metric('accuracy', report['accuracy'])
        mlflow.log_metric('recall_class_1', report['1']['recall'])
        mlflow.log_metric('recall_class_0', report['0']['recall'])
        mlflow.log_metric('f1_score_macro', report['macro avg']['f1-score'])
        
        if 'XGB'in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")

2024/08/14 11:01:51 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly Detection1' does not exist. Creating a new experiment.
2024/08/14 11:01:55 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/14 11:01:56 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression at: http://localhost:8080/#/experiments/218558604920265822/runs/2784ce3f282e4ca5b1064c4147c7ef29.
2024/08/14 11:01:56 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:8080/#/experiments/218558604920265822.
2024/08/14 11:01:58 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/14 11:01:58 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: http://localhost:8080/#/experiments/218558604920265822/runs/a0797522d899444eaf543896c126f

### Register the Model

In [25]:
model_name = 'XGB-smoke'
run_id = input('Please enter RunID')
model_uri = f'runs:/{run_id}/model_name'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Successfully registered model 'XGB-smoke'.
2024/08/14 11:02:21 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-smoke, version 1
Created version '1' of model 'XGB-smoke'.
2024/08/14 11:02:21 INFO mlflow.tracking._tracking_service.client: 🏃 View run XGBClassifier With SMOTE at: http://localhost:8080/#/experiments/218558604920265822/runs/b08462ebda2442758968987be3d60114.
2024/08/14 11:02:21 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:8080/#/experiments/218558604920265822.


### Load the Model

In [27]:
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

MlflowException: The following failures occurred while downloading one or more artifacts from http://localhost:8080/api/2.0/mlflow-artifacts/artifacts/218558604920265822/b08462ebda2442758968987be3d60114/artifacts/model_name:
##### File  #####
API request to http://localhost:8080/api/2.0/mlflow-artifacts/artifacts/218558604920265822/b08462ebda2442758968987be3d60114/artifacts/model_name/ failed with exception HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/218558604920265822/b08462ebda2442758968987be3d60114/artifacts/model_name/ (Caused by ResponseError('too many 500 error responses'))

### Load Model with Aliase name

In [29]:
alias = "champion"
model_uri = f"models:/{model_name}@{alias}"
print(model_uri)

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

models:/XGB-smoke@champion


MlflowException: The following failures occurred while downloading one or more artifacts from http://localhost:8080/api/2.0/mlflow-artifacts/artifacts/218558604920265822/b08462ebda2442758968987be3d60114/artifacts/model_name:
##### File  #####
API request to http://localhost:8080/api/2.0/mlflow-artifacts/artifacts/218558604920265822/b08462ebda2442758968987be3d60114/artifacts/model_name/ failed with exception HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/218558604920265822/b08462ebda2442758968987be3d60114/artifacts/model_name/ (Caused by ResponseError('too many 500 error responses'))

### Transition the Model to Production

In [30]:
current_model_uri = f"models:/{model_name}@champion"
production_model_name = 'anomaly-detection-prod'

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

Successfully registered model 'anomaly-detection-prod'.
Copied version '1' of model 'XGB-smoke' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1723623209937, current_stage='None', description='', last_updated_timestamp=1723623209937, name='anomaly-detection-prod', run_id='b08462ebda2442758968987be3d60114', run_link='', source='models:/XGB-smoke/1', status='READY', status_message='', tags={}, user_id='', version='1'>

In [31]:
model_version = 1
prod_model_uri = f"models:/{production_model_name}@champ"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

MlflowException: The following failures occurred while downloading one or more artifacts from http://localhost:8080/api/2.0/mlflow-artifacts/artifacts/218558604920265822/b08462ebda2442758968987be3d60114/artifacts/model_name:
##### File  #####
API request to http://localhost:8080/api/2.0/mlflow-artifacts/artifacts/218558604920265822/b08462ebda2442758968987be3d60114/artifacts/model_name/ failed with exception HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/218558604920265822/b08462ebda2442758968987be3d60114/artifacts/model_name/ (Caused by ResponseError('too many 500 error responses'))